## Statemachine test

In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(600)
e=dmyplant2.Engine.from_sn(mp,1184199)
# print(f"{e}")
e.dash
# dtrips = e.batch_hist_alarms(p_severities=[600,650,700,800], p_offset=0, p_limit=200000)
# dtrips = dtrips.iloc[::-1] # Alarme umdrehen 
# dtrips.to_pickle('1184199_messages.pkl')
manual = e.batch_hist_alarms(p_severities=[900], p_offset=0, p_limit=200000)
manual = manual.iloc[::-1] # Alarme umdrehen 
manual.to_pickle('1184199_manual.pkl')
manual = pd.read_pickle('1184199_manual.pkl').reset_index()
manual.head()

,index


In [ ]:
from dfsm import msgFSM
messages = pd.read_pickle('1184199_messages.pkl').reset_index()
messages.shape

In [ ]:
# with open('msg.txt', 'w') as f:
#     for index, msg in messages.iterrows():
#         f.write(f"{index:>06} {msg['severity']} {pd.to_datetime(int(msg['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {msg['name']} {msg['message']}\n")

In [ ]:
with open('manual.txt', 'w') as f:
    for index, msg in manual.iterrows():
        f.write(f"{index:>06} {msg['severity']} {pd.to_datetime(int(msg['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {msg['name']} {msg['message']}\n")

In [ ]:
for index, msg in messages[:100000].iterrows():
    if str(msg['message']).startswith('Ready'):
    #if str(msg['message']).startswith('Cold start CPU'):
    #if str(msg['message']).startswith('Service selector'):
        print(f"{index:>06} {msg['severity']} {pd.to_datetime(int(msg['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {msg['name']} {msg['message']}")

In [ ]:
from IPython.display import HTML, display
m = messages[messages.name != '9007'] # filter out hourly messages
m = m[:2000]
display(HTML(m[:5].to_html(escape=False)))

In [ ]:
def Start_FSM(msgs):
    fsmrunner = msgFSM()
    for index,msg in msgs.iterrows():
        #print(msg['message'])
        fsmrunner.send(msg)
    return fsmrunner.completed()

In [ ]:
Start_FSM(m)